In [1]:
import os
import warnings
import sys
import codecs
import numpy as np
import argparse
import json
import pickle

from util import read_passages, evaluate, make_folds, clean_words, test_f1, to_BIO, from_BIO, from_BIO_ind, arg2param

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 1.0
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
sess = tf.Session(config=config)
import keras.backend as K
K.set_session(sess)
from keras.activations import softmax
from keras.regularizers import l2
from keras.models import Model, model_from_json
from keras.layers import Input, LSTM, Dense, Dropout, TimeDistributed, Bidirectional
from keras.callbacks import EarlyStopping,LearningRateScheduler, ModelCheckpoint
from keras.optimizers import Adam, RMSprop, SGD
from crf import CRF
from attention import TensorAttention
from custom_layers import HigherOrderTimeDistributedDense
from generator import BertDiscourseGenerator
from keras_bert import load_trained_model_from_checkpoint, Tokenizer

from discourse_tagger_generator_bert import PassageTagger

import matplotlib.pyplot as plt
from scipy.special import softmax
from matplotlib import transforms

Using TensorFlow backend.


In [2]:
use_attention = True
att_context = "LSTM_clause"
bidirectional = bid = True
crf = True
lstm = False
maxseqlen = 40
maxclauselen = 60
input_size = 768
embedding_dropout=0.4 
high_dense_dropout=0.4
attention_dropout=0.6
lstm_dropout=0.5
word_proj_dim=300 
hard_k=0 
lstm_dim = 350 
rec_hid_dim = 75 
att_proj_dim = 200 
batch_size = 10
reg=0

In [4]:
params = {
    "repfile":"/nas/home/xiangcil/scibert_scivocab_uncased",
    "use_attention": True,
    "batch_size": 10,
    "maxseqlen": 40,
    "maxclauselen": 60
         }
test_file = "test_rct.txt"

In [3]:
prefix="LSTM_clause"
model_ext = "att=%s_cont=%s_lstm=%s_bi=%s_crf=%s"%(str(use_attention), att_context, str(lstm), str(bid), str(crf))
model_config_file = open(prefix+"_config.json", "r")
model_weights_file_name = prefix+"_weights.best.hdf5"
model_label_ind = prefix+"_label_ind.json"
label_ind_json = json.load(open(model_label_ind))
label_ind = {k: int(label_ind_json[k]) for k in label_ind_json}
num_classes = len(label_ind)

In [6]:
nnt = PassageTagger(params)

In [7]:
nnt.tagger = model_from_json(model_config_file.read(), custom_objects={"TensorAttention":TensorAttention, 
    "HigherOrderTimeDistributedDense":HigherOrderTimeDistributedDense,"CRF":CRF})
nnt.tagger.load_weights(model_weights_file_name)

In [8]:
test_seq_lengths, test_generator = nnt.make_data(test_file, label_ind=label_ind, train=True)

Filtering data


In [ ]:
pred_probs, pred_label_seqs, _ = nnt.predict(test_generator, test_seq_lengths)

In [ ]:
outfile = open("test_RCT_BioBERT_LSTM_clause.out","w")
pred_label_seqs = from_BIO(pred_label_seqs)
for pred_label_seq in pred_label_seqs:
    for pred_label in pred_label_seq:
        print(pred_label,file = outfile)
    print("",file = outfile)
outfile.close()